# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Experiment,Environment
import pandas as pd
from azureml.core.dataset import Dataset

In [2]:


ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="automl-run-1")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-186279
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-186279


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# Choose a name for your CPU cluster
cpu_cluster_name = "compute-test"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
filepath="https://raw.githubusercontent.com/soorya-venkatesh/nd00333-capstone/master/starter_file/Telco_customer_churn.csv"
ds = TabularDatasetFactory.from_delimited_files(path = filepath)

df=ds.to_pandas_dataframe()
df.head(3)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,True,Mailed check,53.85,108.15,True,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,True,Electronic check,70.70,151.65,True,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,True,Electronic check,99.65,820.50,True,1,86,5372,Moved


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:

df['Total Charges']=pd.to_numeric(df['Total Charges'], errors='coerce')
df['Total Charges'].fillna(df['Total Charges'].median(),inplace=True)
df.drop(['CustomerID' ,'Count' ,'Country' ,'State' ,'Lat Long' ,'Latitude' ,'Longitude','Zip Code','Churn Label','Churn Score','Streaming TV', 'Streaming Movies','Tenure Months', 'CLTV', 'Churn Reason'], axis=1, inplace=True)

df.to_csv('cleaned_data1.csv', index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./cleaned_data1.csv'])

ds = Dataset.Tabular.from_delimited_files([(datastore, 'cleaned_data1.csv')])

Uploading an estimated of 1 files
Uploading ./cleaned_data1.csv
Uploaded ./cleaned_data1.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [8]:
df.columns

Index(['City', 'Gender', 'Senior Citizen', 'Partner', 'Dependents',
       'Phone Service', 'Multiple Lines', 'Internet Service',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Contract', 'Paperless Billing', 'Payment Method', 'Monthly Charges',
       'Total Charges', 'Churn Value'],
      dtype='object')

In [30]:
print(df['Monthly Charges'].dtype,df['Tenure Months'].dtype,df['Total Charges'].dtype)

float64 int64 float64


In [10]:
from azureml.train.automl import AutoMLConfig
automl_settings = {
       "n_cross_validations": 3,
       "primary_metric": 'AUC_weighted',
       "enable_early_stopping": True,
       "experiment_timeout_minutes": 15,
       "max_concurrent_iterations": 6
   }

# automl config 
automl_config = AutoMLConfig(task = "classification",
                               compute_target = cpu_cluster,
                               training_data = ds,
                               label_column_name = "Churn Value",
                               **automl_settings
                            )

In [11]:
# TODO: Submit your experiment

remote_run = exp.submit(automl_config,show_output=True)


Submitting remote run.
No run_configuration provided, running on compute-test with default configuration
Running on remote compute: compute-test


Experiment,Id,Type,Status,Details Page,Docs Page
automl-run-1,AutoML_dc879c9a-5ebe-458b-a4a9-9a81f456c7f2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTI

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
import joblib
# Retrieve and save your best automl model.
best_model, pipeline = remote_run.get_output()

joblib.dump(pipeline, "automl.joblib")

['automl.joblib']

In [16]:
best_model

Experiment,Id,Type,Status,Details Page,Docs Page
automl-run-1,AutoML_dc879c9a-5ebe-458b-a4a9-9a81f456c7f2_38,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [20]:
best_model = remote_run.register_model(model_name='best_model_automl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
